In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
df=pd.read_csv("/kaggle/input/glass/glass.csv")

In [ ]:
# final output categories 
df.Type.unique()

In [ ]:
df.describe()

In [ ]:
X=["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe"]
y=["Type"]

In [ ]:
X=df[X]
y=df[y]
y=y.sub(1)
X=np.array(X)
y=np.array(y)
X=torch.FloatTensor(X)
y=torch.LongTensor(y)

In [ ]:
tl=TensorDataset(X,y)
dl=DataLoader(tl,shuffle=True,batch_size=128)


In [ ]:
model=nn.Linear(9,7)
loss=nn.CrossEntropyLoss()

In [ ]:
for i,j in dl:
    op=model(i)
    break

In [ ]:
l=j.squeeze()
# print(loss(op,l))
print(op[0])
print(F.softmax(op[0]))

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class Model(nn.Module):
    def __init__(self,ip_size,hi_size,op_size):
        super().__init__()
        self.linear1=nn.Linear(ip_size,32)
        self.linear2=nn.Linear(32,hi_size)
        self.relu=nn.ReLU()
        self.linear3=nn.Linear(hi_size,op_size)
    def forward(self,x):
        x=self.linear1(x)
        x=self.relu(x)
        x=self.linear2(x)
        x=self.relu(x)
        x=self.linear3(x)
        return x

In [ ]:
def fit(epochs,lr,model,loss,optim,dl):
    optim=torch.optim.SGD(model.parameters(),lr=lr,weight_decay=0.1)
    for epoch in range(epochs):
        for i,j in dl:
            op=model(i)
            j=j.squeeze()
            ls=loss(op,j)
            ls.backward()
            optim.step()
            acc=accuracy(op,j)
        print(f"epoch.no={epoch} loss={ls} acc={acc}")
            

In [ ]:
model=Model(9,8,7)
loss=nn.CrossEntropyLoss()
fit(50,1e-8,model,loss,optim,dl)

In [ ]:
for i,j in dl:
    op=model(i)
    l=j.squeeze()
    break

In [ ]:
_,p=torch.max(op,dim=1)
p

In [ ]:
l